In [ ]:
pin_code = 201301

import pandas as pd
import datetime
import requests
import webbrowser
from time import sleep
from pandas.io.json import json_normalize
from IPython.display import display, HTML

youtube_alert = 'https://www.youtube.com/watch?v=io-oreIAuTM&ab_channel=MinistryofHealth%26FamilyWelfare'
cowin ='https://selfregistration.cowin.gov.in/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

In [ ]:
def get_data(pincode, date):
    try:
        data = requests.get(f"https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={pincode}&date={date}", headers=headers).json()
        df = pd.json_normalize(data=data['centers'], record_path='sessions', meta=['name','address','district_name','pincode'],errors='ignore')
    except:
        print('Link not working - Exception Found - auto-wait for 60 seconds to check again')
        sleep(60)
        df = pd.DataFrame()
    
    return df

In [ ]:
def check_availability(pin_code, age_limit=18, minimum_availability = 1):
    
    date = datetime.datetime.today().strftime('%d-%m-%Y')
    df = get_data(pin_code, date)
    if df.shape[0] > 0:
        df.drop(columns=['session_id','slots'], axis=1, inplace=True)
        df.date = pd.to_datetime(df.date, format='%d-%m-%Y')
        available = df[(df.min_age_limit == age_limit) & (df.available_capacity >= minimum_availability)].sort_values(by=['date','available_capacity']).reset_index(drop=True)
        return available
    
    else: return pd.DataFrame()

In [ ]:
i = 1

while 1:
    
    time_now = datetime.datetime.now().strftime('%H:%M:%S')
    df_available = check_availability(pin_code)
    
    if (df_available.shape[0] > 0):
        webbrowser.open(cowin, new=2)
        webbrowser.open(youtube_alert, new=2)
        print(time_now)
        display(df_available)
        sleep(300)    #in seconds
#         break
    else:
        if i%100 == 0:
            print(f'Checking.... {i} at {time_now}')
        i = i + 1
        sleep(3)    #in seconds